# 多層感知器從頭實現

安裝套件庫

In [42]:
import numpy as np
import tensorflow as tf

加載Fashion-MNIST資料集

In [43]:
fashion_mnist = tf.keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

將資料集預處理

In [53]:
# 將影像像素值縮放到0 ~ 1 之間
train_images = train_images / 255.0
test_images = test_images / 255.0
# 將影像轉換成整數資料類型
train_labels = train_labels.astype(np.int32)
test_labels = test_labels.astype(np.int32)

創建訓練集和測試集的批次資料集

In [54]:
# 批次大小
batch_size = 256
# 將訓練集製作成tensorflow 資料集
train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels)).batch(batch_size)
# 將測試集製作成tensorflow 資料集
test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels)).batch(batch_size)

定義參數值

In [55]:
# 定義輸入層、輸出層、隱藏層
num_inputs, num_outputs, num_hiddens = 784, 10, 256
# 定義權重和偏權值並隨機初始化
W1 = tf.Variable(tf.random.normal(shape=(num_inputs, num_hiddens), mean=0, stddev=0.01, dtype=tf.float64))
b1 = tf.Variable(tf.zeros(num_hiddens, dtype=tf.float64))
W2 = tf.Variable(tf.random.normal(shape=(num_hiddens, num_outputs), mean=0, stddev=0.01, dtype=tf.float64))
b2 = tf.Variable(tf.zeros(num_outputs, dtype=tf.float64))

# 這裡將模型參數儲存在一個列表內
params = [W1, b1, W2, b2]

**定義relu函數**

In [56]:
def relu(X):
    return tf.math.maximum(X, 0)

前向傳播的過程

In [48]:
def net(X):
    # 先將輸入值做形狀調整
    X = tf.reshape(X, (-1, num_inputs))
    # 隱藏層輸出
    H = relu(tf.matmul(X, W1) + b1)
    return tf.matmul(H, W2) + b2

定義損失函數

In [49]:
def loss(y_hat, y):
    # 稀疏分類交叉熵
    return tf.losses.sparse_categorical_crossentropy(
        y, y_hat, from_logits=True)

開始訓練，並定義訓練參數

In [50]:
# 定義回合數和學習參數
num_epochs, learning_rate = 10, 0.1
# 定義優化器
optimizer = tf.optimizers.SGD(learning_rate=learning_rate)
# 開始訓練
for epoch in range(num_epochs):
    for X, y in train_dataset:
        # 自動計算梯度
        with tf.GradientTape() as tape:
            # 獲取隱藏層輸出
            y_hat = net(X)
            # 獲取損失值
            l = loss(y_hat, y)
        # 計算損失對於模型參數的梯度
        grads = tape.gradient(l, [W1, b1, W2, b2])
        # 使用優化器來更新模型參數
        optimizer.apply_gradients(zip(grads, [W1, b1, W2, b2]))


定義精確度涵式

In [51]:
def evaluate_accuracy(data_iter, net):
    # 定義初始精確度和樣本數
    acc_sum, n = 0.0, 0
    # 迭代批次資料集
    for X, y in data_iter:
        # 轉換影像資料類型
        y = tf.cast(y, dtype=tf.int64)
        # 計算批次中的準確預測數量
        acc_sum += np.sum(tf.cast(tf.argmax(net(X), axis=1), dtype=tf.int64) == y)
        # 計算總樣本數
        n += y.shape[0]
    return acc_sum / n

In [52]:
test_acc = evaluate_accuracy(test_dataset, net)
print(f'Test accuracy: {test_acc * 100:.2f}%')

Test accuracy: 10.00%
